In [1]:
import pandas as pd
import os

# Define o caminho para a pasta 'raw' dos dados
# O '../' é para subir um nível do diretório 'notebooks' para o diretório raiz do projeto,
# e então descer para 'data/raw'
RAW_DATA_PATH = '../data/raw/'

# Lista dos arquivos CSV
csv_files = {
    'accounts': 'accounts.csv',
    'data_dictionary': 'data_dictionary.csv',
    'products': 'products.csv',
    'sales_pipeline': 'sales_pipeline.csv',
    'sales_teams': 'sales_teams.csv'
}

# Dicionário para armazenar os DataFrames
dfs = {}

# Carrega cada arquivo CSV em um DataFrame e armazena no dicionário
for name, file_name in csv_files.items():
    file_path = os.path.join(RAW_DATA_PATH, file_name)
    try:
        dfs[name] = pd.read_csv(file_path)
        print(f"DataFrame '{name}' carregado com sucesso! Shape: {dfs[name].shape}")
    except FileNotFoundError:
        print(f"Erro: Arquivo '{file_name}' não encontrado em {file_path}")
    except Exception as e:
        print(f"Erro ao carregar '{file_name}': {e}")

print("\n--- Carregamento de dados concluído ---")

DataFrame 'accounts' carregado com sucesso! Shape: (85, 7)
DataFrame 'data_dictionary' carregado com sucesso! Shape: (21, 3)
DataFrame 'products' carregado com sucesso! Shape: (7, 3)
DataFrame 'sales_pipeline' carregado com sucesso! Shape: (8800, 8)
DataFrame 'sales_teams' carregado com sucesso! Shape: (35, 3)

--- Carregamento de dados concluído ---


In [3]:
print("\n--- Primeiras linhas de sales_pipeline ---")
print(dfs['sales_pipeline'].head())


--- Primeiras linhas de sales_pipeline ---
  opportunity_id      sales_agent         product  account deal_stage  \
0       1C1I7A6R      Moses Frase  GTX Plus Basic  Cancity        Won   
1       Z063OYW0  Darcel Schlecht          GTXPro    Isdom        Won   
2       EC4QE1BX  Darcel Schlecht      MG Special  Cancity        Won   
3       MV1LWRNH      Moses Frase       GTX Basic  Codehow        Won   
4       PE84CX4O        Zane Levy       GTX Basic   Hatfan        Won   

  engage_date  close_date  close_value  
0  2016-10-20  2017-03-01       1054.0  
1  2016-10-25  2017-03-11       4514.0  
2  2016-10-25  2017-03-07         50.0  
3  2016-10-25  2017-03-09        588.0  
4  2016-10-25  2017-03-02        517.0  


In [4]:
print("\n--- Informações de sales_pipeline ---")
dfs['sales_pipeline'].info()


--- Informações de sales_pipeline ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8800 entries, 0 to 8799
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   opportunity_id  8800 non-null   object 
 1   sales_agent     8800 non-null   object 
 2   product         8800 non-null   object 
 3   account         7375 non-null   object 
 4   deal_stage      8800 non-null   object 
 5   engage_date     8300 non-null   object 
 6   close_date      6711 non-null   object 
 7   close_value     6711 non-null   float64
dtypes: float64(1), object(7)
memory usage: 550.1+ KB


In [5]:
print("\n--- Estatísticas descritivas de sales_pipeline ---")
print(dfs['sales_pipeline'].describe())


--- Estatísticas descritivas de sales_pipeline ---
        close_value
count   6711.000000
mean    1490.915512
std     2320.670773
min        0.000000
25%        0.000000
50%      472.000000
75%     3225.000000
max    30288.000000


In [6]:
print("\n--- Contagem de valores para a coluna 'deal_stage' ---")
print(dfs['sales_pipeline']['deal_stage'].value_counts())


--- Contagem de valores para a coluna 'deal_stage' ---
deal_stage
Won            4238
Lost           2473
Engaging       1589
Prospecting     500
Name: count, dtype: int64


In [7]:
# Importar plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- 1. Criação da Variável Alvo (Target) ---
# Mapear 'Lost' para 1 (Oportunidade Perdida) e o restante para 0 (Oportunidade Não Perdida)
# Criar uma cópia para não modificar o DataFrame original diretamente antes de explorar
df_sales_pipeline_eda = dfs['sales_pipeline'].copy()

df_sales_pipeline_eda['target'] = df_sales_pipeline_eda['deal_stage'].apply(lambda x: 1 if x == 'Lost' else 0)

print("Contagem de oportunidades perdidas (1) vs. não perdidas (0):")
print(df_sales_pipeline_eda['target'].value_counts())
print(f"Proporção de oportunidades perdidas: {df_sales_pipeline_eda['target'].value_counts(normalize=True)[1]:.2%}")


Contagem de oportunidades perdidas (1) vs. não perdidas (0):
target
0    6327
1    2473
Name: count, dtype: int64
Proporção de oportunidades perdidas: 28.10%


In [8]:
# --- 2. Conversão de Tipos de Dados (Datas) ---
# Converter colunas de data para datetime
df_sales_pipeline_eda['engage_date'] = pd.to_datetime(df_sales_pipeline_eda['engage_date'], errors='coerce')
df_sales_pipeline_eda['close_date'] = pd.to_datetime(df_sales_pipeline_eda['close_date'], errors='coerce')

print("\nTipos de dados após conversão de datas:")
df_sales_pipeline_eda.info()


Tipos de dados após conversão de datas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8800 entries, 0 to 8799
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   opportunity_id  8800 non-null   object        
 1   sales_agent     8800 non-null   object        
 2   product         8800 non-null   object        
 3   account         7375 non-null   object        
 4   deal_stage      8800 non-null   object        
 5   engage_date     8300 non-null   datetime64[ns]
 6   close_date      6711 non-null   datetime64[ns]
 7   close_value     6711 non-null   float64       
 8   target          8800 non-null   int64         
dtypes: datetime64[ns](2), float64(1), int64(1), object(5)
memory usage: 618.9+ KB


In [ ]:
# --- 3. Plot: Distribuição da Variável Alvo ---
# Captura o resultado do value_counts e reseta o índice, nomeando as colunas
target_counts = df_sales_pipeline_eda['target'].value_counts(normalize=True).reset_index()
target_counts.columns = ['target_status', 'proportion'] # Renomeando as colunas para clareza

fig = px.bar(target_counts,
             x='target_status', y='proportion',
             title='Distribuição da Variável Alvo (Oportunidades Perdidas vs. Não Perdidas)',
             labels={'target_status': 'Status da Oportunidade', 'proportion': 'Proporção'},
             color='target_status',
             color_discrete_map={0: 'lightgreen', 1: 'salmon'},
             text_auto='.2%') # Exibir o percentual direto na barra

fig.update_layout(xaxis_title="0: Não Perdida (Won, Engaging, Prospecting) | 1: Perdida (Lost)",
                  yaxis_title="Proporção")
fig.show()

In [11]:
dfs['data_dictionary'].head()

,Table,Field,Description
0,accounts,account,Company name
1,accounts,sector,Industry
2,accounts,year_established,Year Established
3,accounts,revenue,Annual revenue (in millions of USD)
4,accounts,employees,Number of employees


In [12]:
dfs['data_dictionary'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Table        21 non-null     object
 1   Field        21 non-null     object
 2   Description  21 non-null     object
dtypes: object(3)
memory usage: 636.0+ bytes


In [13]:
dfs['data_dictionary'].describe()

,Table,Field,Description
count,21,21,21
unique,4,18,19
top,sales_pipeline,account,Company name
freq,8,2,2


In [15]:
# --- 4. Análise RFM (Recência, Frequência, Valor Monetário) ---
print("\n--- Iniciando Análise RFM ---")

# 4.1. Filtrar apenas oportunidades GANHAS ('Won')
# A análise RFM foca em clientes que já realizaram transações
df_won_opportunities = df_sales_pipeline_eda[df_sales_pipeline_eda['deal_stage'] == 'Won'].copy()

# Remover linhas onde close_date ou close_value são nulos para RFM, pois não são transações completas
df_won_opportunities.dropna(subset=['close_date', 'close_value', 'account'], inplace=True)

# Garantir que close_value é numérico e > 0 para RFM
df_won_opportunities['close_value'] = pd.to_numeric(df_won_opportunities['close_value'], errors='coerce')
df_won_opportunities = df_won_opportunities[df_won_opportunities['close_value'] > 0]


if not df_won_opportunities.empty:
    # 4.2. Calcular Recência, Frequência, Valor Monetário
    # A data de referência é a data mais recente de fechamento no dataset
    # (ou a data atual se estivéssemos em produção e a data atual fosse mais recente que o dataset)
    snapshot_date = df_won_opportunities['close_date'].max() + pd.Timedelta(days=1) # Um dia após a última venda

    rfm_df = df_won_opportunities.groupby('account').agg(
        Recency=('close_date', lambda date: (snapshot_date - date.max()).days),
        Frequency=('opportunity_id', 'count'),
        Monetary=('close_value', 'sum')
    ).reset_index()

    print("\n--- DataFrame RFM (primeiras 5 linhas) ---")
    print(rfm_df.head())

    print("\n--- Estatísticas descritivas do DataFrame RFM ---")
    print(rfm_df.describe())

    # 4.3. Visualização RFM (Exemplo: Histogramas das distribuições)
    fig = make_subplots(rows=1, cols=3, subplot_titles=("Recência (Dias)", "Frequência (Oportunidades Ganhas)", "Valor Monetário Total"))

    fig.add_trace(go.Histogram(x=rfm_df['Recency'], name='Recência'), row=1, col=1)
    fig.add_trace(go.Histogram(x=rfm_df['Frequency'], name='Frequência'), row=1, col=2)
    fig.add_trace(go.Histogram(x=rfm_df['Monetary'], name='Monetário'), row=1, col=3)

    fig.update_layout(title_text='Distribuição das Métricas RFM', height=400, showlegend=False)
    fig.show()

    # Opcional: Segmentação RFM básica (ex: quintis)
    # rfm_df['R_score'] = pd.qcut(rfm_df['Recency'], 5, labels=False, duplicates='drop')
    # rfm_df['F_score'] = pd.qcut(rfm_df['Frequency'], 5, labels=False, duplicates='drop')
    # rfm_df['M_score'] = pd.qcut(rfm_df['Monetary'], 5, labels=False, duplicates='drop')
    # print("\n--- RFM Scores (primeiras 5 linhas) ---")
    # print(rfm_df.head())

else:
    print("Não há oportunidades 'Won' suficientes para realizar a análise RFM após filtragem.")


--- Iniciando Análise RFM ---

--- DataFrame RFM (primeiras 5 linhas) ---
            account  Recency  Frequency  Monetary
0  Acme Corporation        5         34  101744.0
1        Betasoloin        4         34   97036.0
2          Betatech        3         53  107408.0
3        Bioholding        4         50   90991.0
4           Bioplex       16         31   67393.0

--- Estatísticas descritivas do DataFrame RFM ---
         Recency   Frequency       Monetary
count  85.000000   85.000000      85.000000
mean    5.129412   49.858824  117712.164706
std     5.098525   18.538347   46525.656119
min     1.000000   23.000000   51632.000000
25%     2.000000   36.000000   85047.000000
50%     4.000000   47.000000  111533.000000
75%     7.000000   57.000000  140086.000000
max    33.000000  115.000000  341455.000000
